# 5. 결측치 방법론 중 각 카테고리에 맞는 방법론 찾는 코드

- `../data/카테고리이름/` 위치에 결측치처리한 데이터 파일 있음
- 파일 저장위치 : 아래 코드 실행하면 각 카테고리별 결측치 방법 모델링 결과가 `../data/evaluation/` 에 저장
- 결과 확인하고 카테고리에 맞는 결측치 방법론 선택 (최종 : missing값 대체 선택)

- 연관성 뒤에 나오는 Label Encoding 부분 부터 실행

- 방법론 결정 후 `6.1_LG_고객데이터_Calibration_XAI.ipynb` 파일에서 카테고리별 모델링 예측 결과 확인 가능 + calibration 결과 포함

In [ ]:
import os
import pandas as pd
from tqdm.auto import tqdm
from datetime import datetime
from sklearn.model_selection import train_test_split

# 분류 알고리즘
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
# from lightgbm import LGBMClassifier
# from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

# 분류용 평가 함수
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

from sklearn.preprocessing import StandardScaler

# import joblib
# import pickle
pd.set_option('display.max_columns', None)

In [ ]:
# !pip install xgboost
# !pip install libomp # didn't work
# !conda install lightgbm --y # worked

## multicollinearity

In [ ]:
multi_df = air_df[['고객ID', '멤버십고객여부', '케어솔루션품목수', '케어십가입여부', '케어십품목수', '최근3년구매금액',
       '최근구매채널', 'Only&Best품목수', '건강관리가전품목수', '연령대', '주거형태', '아파트시세', '아파트평형대',
       '포인트사용빈도', '보유멤버십포인트', '멤버십가입경로', '제휴카드보유여부', '프리미엄카드보유여부', '제휴가입여부',
       '멤버십앱설치여부', '거주광역명', '거주시군구명', 'label_y', '총구매빈도', '최대구매가격대',
       '최근매출일로부터경과일', '주구매제품카테고리']]

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# from scipy.stats import chi2_contingency
from tqdm import tqdm

# Cramér's V 통계량을 계산하는 함수
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2_corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    r_corr = r - ((r-1)**2)/(n-1)
    k_corr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2_corr / min((k_corr-1), (r_corr-1)))

# 데이터프레임 로드 (여기서는 예시로 'df'라고 가정)
# df = pd.read_csv('your_file.csv')

# 범주형 변수들의 목록
categorical_columns = multi_df.select_dtypes(include=['object', 'category']).columns

# 모든 범주형 변수 쌍에 대한 Cramér's V 계산
cramers_v_matrix = pd.DataFrame(index=categorical_columns, columns=categorical_columns, dtype=float)
for col in tqdm(categorical_columns):
    for row in categorical_columns:
        cramers_v_matrix.at[col, row] = cramers_v(multi_df[col], multi_df[row])


In [ ]:
cramers_v_matrix

In [ ]:
# Cramér's V 행렬 시각화
plt.rcParams['font.family'] = 'AppleGothic'
plt.figure(figsize=(10, 8))
sns.heatmap(cramers_v_matrix, annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Cramér's V between Categorical Variables")
plt.show() 

In [ ]:
# # 높은 연관성을 가진 변수 쌍 식별
# highly_correlated_pairs = set()
# threshold = 0.9  # 임계값 설정
# for col in cramers_v_matrix.columns:
#     for row in cramers_v_matrix.index:
#         if (cramers_v_matrix.at[col, row] > threshold) and (col != row):
#             highly_correlated_pairs.add(frozenset((col, row)))

# # 연관성이 높은 변수 중 하나를 제거
# # 이 부분은 연관성이 높은 변수 쌍 중 어떤 변수를 제거할지 결정하는 로직이 필요합니다.
# # 예를 들어, 각 쌍에서 임의로 하나를 제거할 수 있습니다.
# for pair in highly_correlated_pairs:
#     pair = list(pair)
#     df_merge.drop(pair[1], axis=1, inplace=True)

## 연속형 multicollinearity

In [ ]:
import pandas as pd
# import statsmodels.api as sm


def multicol_continue(df):
    # 독립 변수만 선택합니다.
    X = df.select_dtypes(include=['number'])  # 숫자형 변수만 선택

    # 각 독립 변수에 대한 VIF 계산
    vif_data = pd.DataFrame()
    vif_data["Variable"] = X.columns
    vif_data["VIF"] = [1 / (1 - sm.OLS(X[col], X.drop(col, axis=1)).fit().rsquared) for col in X.columns]

    return(vif_data)


In [ ]:
multi_df = air_df[['고객ID', '멤버십고객여부', '케어솔루션품목수', '케어십가입여부', '케어십품목수', '최근3년구매금액',
       '최근구매채널', 'Only&Best품목수', '건강관리가전품목수', '연령대', '주거형태', '아파트시세', '아파트평형대',
       '포인트사용빈도', '보유멤버십포인트', '멤버십가입경로', '제휴카드보유여부', '프리미엄카드보유여부', '제휴가입여부',
       '멤버십앱설치여부', '거주광역명', '거주시군구명', 'label_y', '총구매빈도', '최대구매가격대',
       '최근매출일로부터경과일', '주구매제품카테고리']]

In [ ]:
multicol_continue(multi_df)

In [ ]:
# VIF 보고 5 이상인 경우 제거 
# 제거한 컬럼 순서 (총구매빈도 -> 최근구매채널 -> 케어십가입여부)

In [ ]:
# multi_df.drop('케어십가입여부', axis = 1, inplace = True)

## Label Encoding

In [2]:
# list(air_df['보유멤버십포인트'].unique())

label_list1 = ['0',  '1만↓', '3만↓', '5만↓', '10만↓','20만↓', '30만↓', '30만↑']

label_dict1 = {value: index for index, value in enumerate(label_list1)}

In [3]:
# list(air_df['최근3년구매금액'].unique())
label_list2 = ['0원', '1KK미만', '1KK이상3KK미만','3KK이상5KK미만',  '5KK이상10KK미만','10KK이상15KK미만',
 '15KK이상20KK미만','20KK이상25KK미만','25KK이상30KK미만', '30KK↑']

label_dict2 = {value: index for index, value in enumerate(label_list2)}

In [4]:
# list(air_df['포인트사용빈도'].unique())
label_list3 = ['미사용','1~2회', '3~4회', '5~6회','7~8회','9~10회','11~12회','13~14회','15~16회','17~18회','19~20회', '20회↑']

label_dict3 = {value: index for index, value in enumerate(label_list3)}

In [5]:
# list(air_df['아파트평형대'].unique())
label_list4 = ['10평대↓', '10평대', '20평대', '30평대', '40평대', '50평대', '60평대','70평대↑']

label_dict4 = {value: index for index, value in enumerate(label_list4)}

In [6]:
# list(air_df['아파트시세'].unique())
label_list5 = [ '1억↓', '1~2억', '3~4억', '5~6억', '7~8억', '9~10억', '11~15억', '16~29억','30억↑']

label_dict5 = {value: index for index, value in enumerate(label_list5)}


In [7]:
# air_df.isnull().sum()

In [8]:
# list(air_df['금액'].unique())
label_list6 = ['', '0만원 이상 100만원 이하', '100만원 이상 200만원 이하', '200만원 이상 300만원 이하', 
               '300만원 이상 400만원 이하', '400만원 이상 500만원 이하', '500만원 이상 600만원 이하',
       '600만원 이상 700만원 이하', '700만원 이상 800만원 이하', '800만원 이상 900만원 이하', '900만원 이상 1000만원 이하', '1000만원 이상']

label_dict6 = {value: index for index, value in enumerate(label_list6)}

In [9]:
# list(air_df['월'].unique())
label_list7 = ['', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

label_dict7 = {value: index for index, value in enumerate(label_list7)}

In [10]:
# list(health_df['연령대'].unique())
label_list8 = ['10대', '20대', '30대', '40대', '50대', '60대(60~64)', '60대(65~69)', '70대', '80대↑']

label_dict8 = {value: index for index, value in enumerate(label_list8)}

In [11]:
label_list14 = ['Missing', '10평대↓', '10평대', '20평대', '30평대', '40평대', '50평대', '60평대','70평대↑']
label_dict14 = {value: index for index, value in enumerate(label_list14)}

label_list15 = [ 'Missing', '1억↓', '1~2억', '3~4억', '5~6억', '7~8억', '9~10억', '11~15억', '16~29억','30억↑']
label_dict15 = {value: index for index, value in enumerate(label_list15)}

label_list16 = ['Missing', '','0만원 이상 100만원 이하', '100만원 이상 200만원 이하', '200만원 이상 300만원 이하', 
               '300만원 이상 400만원 이하', '400만원 이상 500만원 이하', '500만원 이상 600만원 이하',
       '600만원 이상 700만원 이하', '700만원 이상 800만원 이하', '800만원 이상 900만원 이하', '900만원 이상 1000만원 이하', '1000만원 이상']

label_dict16 = {value: index for index, value in enumerate(label_list16)}


label_list18 = ['Missing', '10대', '20대', '30대', '40대', '50대', '60대(60~64)', '60대(65~69)', '70대', '80대↑']
label_dict18 = {value: index for index, value in enumerate(label_list18)}

In [12]:
def mapping_missing(df):
    for i in range(1, 9):
        if f'금액_{i}' in df.columns:
            df[f'금액_{i}'] = df[f'금액_{i}'].map(label_dict16)

    for i in range(1, 9):
        if f'월_{i}' in df.columns:
            df[f'월_{i}'] = df[f'월_{i}'].map(label_dict7)

    df['보유멤버십포인트'] = df['보유멤버십포인트'].map(label_dict1)
    df['최근3년구매금액'] = df['최근3년구매금액'].map(label_dict2)
    df['포인트사용빈도'] = df['포인트사용빈도'].map(label_dict3)
    df['아파트평형대'] = df['아파트평형대'].map(label_dict14)
    df['아파트시세'] = df['아파트시세'].map(label_dict15)
    df['연령대'] = df['연령대'].map(label_dict18)
    return df

def mapping_(df):
    for i in range(1, 9):
        if f'금액_{i}' in df.columns:
            df[f'금액_{i}'] = df[f'금액_{i}'].map(label_dict6)

    for i in range(1, 9):
        if f'월_{i}' in df.columns:
            df[f'월_{i}'] = df[f'월_{i}'].map(label_dict7)

    df['보유멤버십포인트'] = df['보유멤버십포인트'].map(label_dict1)
    df['최근3년구매금액'] = df['최근3년구매금액'].map(label_dict2)
    df['포인트사용빈도'] = df['포인트사용빈도'].map(label_dict3)
    df['아파트평형대'] = df['아파트평형대'].map(label_dict4)
    df['아파트시세'] = df['아파트시세'].map(label_dict5)
    df['연령대'] = df['연령대'].map(label_dict8)
    return df

def mapping_etc(df):
    for i in range(1, 4):
        if f'금액_{i}' in df.columns:
            df[f'금액_{i}'] = df[f'금액_{i}'].map(label_dict6)

    for i in range(1, 4):
        if f'월_{i}' in df.columns:
            df[f'월_{i}'] = df[f'월_{i}'].map(label_dict7)

    df['보유멤버십포인트'] = df['보유멤버십포인트'].map(label_dict1)
    df['최근3년구매금액'] = df['최근3년구매금액'].map(label_dict2)
    df['포인트사용빈도'] = df['포인트사용빈도'].map(label_dict3)
    df['아파트평형대'] = df['아파트평형대'].map(label_dict4)
    df['아파트시세'] = df['아파트시세'].map(label_dict5)
    df['연령대'] = df['연령대'].map(label_dict8)
    return df

## One-hot Encoding, Label Encoding, Scaling, Train-test Split

In [13]:
def split_df(df):
    onehot_col = ['멤버십고객여부', '케어십가입여부',
           '최근구매채널', '주거형태', '멤버십가입경로', '제휴카드보유여부', '프리미엄카드보유여부', '제휴가입여부',
           '멤버십앱설치여부', '거주광역명', '거주시군구명', '타겟제품명_1', '타겟제품명_2', '타겟제품명_3',
                   '타겟제품명_4', '타겟제품명_5', '타겟제품명_6', '타겟제품명_7', '타겟제품명_8', '주구매제품카테고리']
    
    # Convert columns in onehot_col to string
    for col in onehot_col:
        df[col] = df[col].astype(str)

    # 원핫 인코딩
    df_encoded = pd.get_dummies(df[onehot_col])
    
    df_encoded = pd.concat([df_encoded, df[['보유멤버십포인트', '최근3년구매금액', '포인트사용빈도', '아파트평형대', '아파트시세',
                                            '건강관리가전품목수', '총구매빈도', '케어솔루션품목수', '케어십품목수', 
                                            'Only&Best품목수', '최대구매가격대', '연령대',
                                            '금액_1', '금액_2', '금액_3', '금액_4', '금액_5', '금액_6', '금액_7', '금액_8',
                                            '년도_1', '년도_2', '년도_3', '년도_4', '년도_5', '년도_6', '년도_7', '년도_8', 
                                            '월_1', '월_2', '월_3', '월_4', '월_5', '월_6', '월_7', '월_8', 
                                            'label_y']]], axis = 1)
    
    X = df_encoded.drop('label_y', axis=1)
    y = df_encoded['label_y']
    column_names = X.columns
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    X_scaled_df = pd.DataFrame(X_scaled, columns=column_names)
    
    X_train, X_val, y_train, y_val = train_test_split(X_scaled_df, y, test_size=0.25, random_state = 42)
    
    return X_train, X_val, y_train, y_val


def split_df_etc(df):
    onehot_col = ['멤버십고객여부', '케어십가입여부',
           '최근구매채널', '주거형태', '멤버십가입경로', '제휴카드보유여부', '프리미엄카드보유여부', '제휴가입여부',
           '멤버십앱설치여부', '거주광역명', '거주시군구명', '타겟제품명_1', '타겟제품명_2', '타겟제품명_3', '주구매제품카테고리']
    
    # Convert columns in onehot_col to string
    for col in onehot_col:
        df[col] = df[col].astype(str)

    # 원핫 인코딩
    df_encoded = pd.get_dummies(df[onehot_col])
    
    df_encoded = pd.concat([df_encoded, df[['보유멤버십포인트', '최근3년구매금액', '포인트사용빈도', '아파트평형대', '아파트시세',
                                            '건강관리가전품목수', '총구매빈도', '케어솔루션품목수', '케어십품목수', 
                                            'Only&Best품목수', '최대구매가격대', '연령대',
                                            '금액_1', '금액_2', '금액_3',
                                            '년도_1', '년도_2', '년도_3',  
                                            '월_1', '월_2', '월_3', 
                                            'label_y']]], axis = 1)
    
    
    X = df_encoded.drop('label_y', axis=1)
    y = df_encoded['label_y']
    column_names = X.columns
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    X_scaled_df = pd.DataFrame(X_scaled, columns=column_names)
    
    X_train, X_val, y_train, y_val = train_test_split(X_scaled_df, y, test_size=0.25, random_state = 42)
    
    return X_train, X_val, y_train, y_val




In [14]:
def knn_model_split(df):
    for i in range(1, 9):
        df[f'월_{i}'] = df[f'월_{i}'].fillna(0)
        df[f'금액_{i}'] = df[f'금액_{i}'].fillna(0)
        df[f'년도_{i}'] = df[f'년도_{i}'].fillna(0)
        df[f'타겟제품명_{i}'] = df[f'타겟제품명_{i}'].fillna('')



    label_list8 = ['10대', '20대', '30대', '40대', '50대', '60대(60~64)', '60대(65~69)', '70대', '80대↑']

    label_dict8 = {value: index for index, value in enumerate(label_list8)}
    df['연령대'] = df['연령대'].map(label_dict8)

    onehot_col = [
           '최근구매채널', '주거형태', '멤버십가입경로', 
            '거주광역명', '거주시군구명', '타겟제품명_1', '타겟제품명_2', '타겟제품명_3',
                   '타겟제품명_4', '타겟제품명_5', '타겟제품명_6', '타겟제품명_7', '타겟제품명_8', '주구매제품카테고리']

    # Convert columns in onehot_col to string
    for col in onehot_col:
        df[col] = df[col].astype(str)

    # 원핫 인코딩
    df_encoded = pd.get_dummies(df[onehot_col])

    df_encoded = pd.concat([df_encoded, df[['멤버십고객여부', '케어십가입여부', '제휴카드보유여부', '프리미엄카드보유여부', '제휴가입여부', '보유멤버십포인트', '최근3년구매금액', '포인트사용빈도', '아파트평형대', '아파트시세',
                                            '건강관리가전품목수', '총구매빈도', '케어솔루션품목수', '케어십품목수', '멤버십앱설치여부',
                                            'Only&Best품목수', '최대구매가격대', '연령대',
                                            '금액_1', '금액_2', '금액_3', '금액_4', '금액_5', '금액_6', '금액_7', '금액_8',
                                            '년도_1', '년도_2', '년도_3', '년도_4', '년도_5', '년도_6', '년도_7', '년도_8', 
                                            '월_1', '월_2', '월_3', '월_4', '월_5', '월_6', '월_7', '월_8', 
                                            'label_y']]], axis = 1)

    X = df_encoded.drop('label_y', axis=1)
    y = df_encoded['label_y']
    column_names = X.columns

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    X_scaled_df = pd.DataFrame(X_scaled, columns=column_names)

    X_train, X_val, y_train, y_val = train_test_split(X_scaled_df, y, test_size=0.25, random_state = 42)

    return X_train, X_val, y_train, y_val

def knn_model_split_etc(df):
    for i in range(1, 4):
        df[f'월_{i}'] = df[f'월_{i}'].fillna(0)
        df[f'금액_{i}'] = df[f'금액_{i}'].fillna(0)
        df[f'년도_{i}'] = df[f'년도_{i}'].fillna(0)
        df[f'타겟제품명_{i}'] = df[f'타겟제품명_{i}'].fillna('')



    label_list8 = ['10대', '20대', '30대', '40대', '50대', '60대(60~64)', '60대(65~69)', '70대', '80대↑']

    label_dict8 = {value: index for index, value in enumerate(label_list8)}
    df['연령대'] = df['연령대'].map(label_dict8)

    onehot_col = [
           '최근구매채널', '주거형태', '멤버십가입경로', 
            '거주광역명', '거주시군구명', '타겟제품명_1', '타겟제품명_2', '타겟제품명_3', '주구매제품카테고리']

    # Convert columns in onehot_col to string
    for col in onehot_col:
        df[col] = df[col].astype(str)

    # 원핫 인코딩
    df_encoded = pd.get_dummies(df[onehot_col])

    df_encoded = pd.concat([df_encoded, df[['멤버십고객여부', '케어십가입여부', '제휴카드보유여부', '프리미엄카드보유여부', '제휴가입여부', '보유멤버십포인트', '최근3년구매금액', '포인트사용빈도', '아파트평형대', '아파트시세',
                                            '건강관리가전품목수', '총구매빈도', '케어솔루션품목수', '케어십품목수', '멤버십앱설치여부',
                                            'Only&Best품목수', '최대구매가격대', '연령대',
                                            '금액_1', '금액_2', '금액_3',
                                            '년도_1', '년도_2', '년도_3',  
                                            '월_1', '월_2', '월_3', 
                                            'label_y']]], axis = 1)

    X = df_encoded.drop('label_y', axis=1)
    y = df_encoded['label_y']
    column_names = X.columns

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    X_scaled_df = pd.DataFrame(X_scaled, columns=column_names)

    X_train, X_val, y_train, y_val = train_test_split(X_scaled_df, y, test_size=0.25, random_state = 42)

    return X_train, X_val, y_train, y_val

## Modeling

In [15]:
def modeling_code(category_name, file_name, X_train, y_train, X_val, y_val):

    # 평가 결과를 담을 딕셔너리
    dict1 = {
        'accuracy' : [],
        'precision' : [],
        'recall' : [],
        'f1' : [],
        'roc_auc' : [],
        'name' : [],
        '결측치방법' : []
    }
   
    # 검증할 모델들 생성

    model1 = LogisticRegression(max_iter=1000)
    model2 = DecisionTreeClassifier()
    model3 = RandomForestClassifier(n_estimators=100)
    model4 = AdaBoostClassifier(n_estimators=100)
    model5 = GradientBoostingClassifier(n_estimators=100)

    model_list = [model1, model2, model3, model4, model5]

    # 모델 이름 목록
    model_name = ['LR', 'DT', 'RF', 'Ada', 'GB'] 

    # 학습용 데이터를 학습.
    for m1 in tqdm(model_list) :
        m1.fit(X_train, y_train)

    # 학습용 데이터를 통해 결과를 예측
    pred_list = []

    for m2 in tqdm(model_list) :
        pred = m2.predict(X_val)

        pred_list.append(pred)

    # 각 모델별 예측 결과를 하나씩 추출하여 평가한다.
    for idx, p1 in enumerate(pred_list) :

        r1 = accuracy_score(y_val, p1)
        r2 = precision_score(y_val, p1)
        r3 = recall_score(y_val, p1)
        r4 = f1_score(y_val, p1)
        r5 = roc_auc_score(y_val, p1)

        dict1['accuracy'].append(r1)
        dict1['precision'].append(r2)
        dict1['recall'].append(r3)
        dict1['f1'].append(r4)
        dict1['roc_auc'].append(r5)
        dict1['name'].append(model_name[idx])
        dict1['결측치방법'].append(file_name.split('_')[2])

    df100 = pd.DataFrame(dict1)
#     df100.to_csv(f'../data/evaluation/{category_name}_{file_name.split("_")[2]}_evaluation_20231219.csv', encoding = 'utf-8-sig')

In [16]:
# https://mydevjourney.tistory.com/76 에러가 났을 경우
# pip install threadpoolctl==3.1.0


### Main 

In [29]:
def maincode(category_name):
    # 데이터 폴더 경로 설정
    data_folder = f'../data/{category_name}/'

    # 데이터 폴더 내의 파일 목록 확인
    file_list = os.listdir(data_folder)
    print(file_list)

    # 파일 목록을 돌면서 각 파일을 열어서 mapping 함수 실행
    for file_name in file_list:
        if file_name.endswith('.csv'):  # CSV 파일만 대상으로 함

            method = file_name.split('_')[2]
            if method in ['값생성']:

                file_path = os.path.join(data_folder, file_name)
                df = pd.read_csv(file_path)
                
                print(method)
                if category_name == 'etc':
                    for i in range(1, 4):
                        df.loc[df[f'년도_{i}'].isnull(), f'년도_{i}'] = 0
        
                    df = df.fillna('')
                    df_mapped = mapping_etc(df)
                    X_train, X_val, y_train, y_val = split_df_etc(df_mapped)
                    modeling_code(category_name, file_name, X_train, y_train, X_val, y_val)
                else:   
                    
                    for i in range(1, 9):
                        df.loc[df[f'년도_{i}'].isnull(), f'년도_{i}'] = 0
                        
                    df = df.fillna('')
                    df_mapped = mapping_missing(df)
                    X_train, X_val, y_train, y_val = split_df(df_mapped)
                    modeling_code(category_name, file_name, X_train, y_train, X_val, y_val)
            elif method in ['랜덤', 'f1기준', '최빈값']:
                file_path = os.path.join(data_folder, file_name)
                df = pd.read_csv(file_path)
                print(method)
                if category_name == 'etc':
                    for i in range(1, 4):
#                         df[f'월_{i}'] = df[f'월_{i}'].fillna(0)
                        df.loc[df[f'년도_{i}'].isnull(), f'년도_{i}'] = 0
        
                    df = df.fillna('')
                    df_mapped = mapping_etc(df)
                    X_train, X_val, y_train, y_val = split_df_etc(df_mapped)
                    modeling_code(category_name, file_name, X_train, y_train, X_val, y_val)
                
                else:   
                    
                    for i in range(1, 9):
#                         df[f'월_{i}'] = df[f'월_{i}'].fillna(0)
                        df.loc[df[f'년도_{i}'].isnull(), f'년도_{i}'] = 0
                        
                    df = df.fillna('')
                    df_mapped = mapping_(df)
                    X_train, X_val, y_train, y_val = split_df(df_mapped)
                    modeling_code(category_name, file_name, X_train, y_train, X_val, y_val)
                    
            elif method in ['knndistance', 'knnuniform']:
                file_path = os.path.join(data_folder, file_name)
                df = pd.read_csv(file_path)
                print(method)
                if category_name == 'etc':
                    X_train, X_val, y_train, y_val = knn_model_split_etc(df)
                    modeling_code(category_name, file_name, X_train, y_train, X_val, y_val)
                else:
                    X_train, X_val, y_train, y_val = knn_model_split(df)
                    modeling_code(category_name, file_name, X_train, y_train, X_val, y_val)
        
            else:
                continue


## 실행하면 각 카테고리별 결측치 방법 모델링 결과가 `../data/evaluation/` 에 저장

- 가장 좋은 결측치 방법으로 선택해서 사용
- 결론 : missing값 대체 방법 사용

In [30]:
maincode('clean')

['save_model', 'clean_df_값생성_2023제거.csv', 'clean_df_knndistance_2023제거.csv', '.DS_Store', 'clean_df_랜덤_2023제거.csv', 'clean_df_최빈값_2023제거.csv', 'clean_df_f1기준_2023제거.csv', 'image', 'clean_df_knnuniform_2023제거.csv', 'metric']


/var/folders/np/0qvclldj141855670r2yt4g80000gp/T/ipykernel_52009/236761990.py:17: DtypeWarning: Columns (7,54,55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


값생성


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

/var/folders/np/0qvclldj141855670r2yt4g80000gp/T/ipykernel_52009/236761990.py:64: DtypeWarning: Columns (54) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


knnuniform


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
maincode('air')

In [88]:
maincode('health')

../data/health/health_df_값생성_2023제거.csv
값생성


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [89]:
maincode('display')

../data/display/display_df_값생성_2023제거.csv


/var/folders/gr/n8_lnq3n4bq1z2j7w3hrtg_40000gn/T/ipykernel_98762/432456705.py:1: DtypeWarning: Columns (7,54,55) have mixed types.Specify dtype option on import or set low_memory=False.
  maincode('display')


값생성


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [90]:
maincode('Kitchen')

../data/Kitchen/kitchen_df_값생성_2023제거.csv


/var/folders/gr/n8_lnq3n4bq1z2j7w3hrtg_40000gn/T/ipykernel_98762/4221296798.py:1: DtypeWarning: Columns (7,46,47,50,51,54,55) have mixed types.Specify dtype option on import or set low_memory=False.
  maincode('Kitchen')


값생성


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [92]:
maincode('etc')

In [46]:
# maincode('all')